In [1]:
# scraping libraries to read html code
import requests
from bs4 import BeautifulSoup
import json

In [2]:
# pandas library to create dataframe
# re for match
import pandas as pd
import numpy as np
import re

In [3]:
# standard library to count frequency 
import collections 

In [15]:
# Direct to initial results
url = 'https://www.yelp.com/search?find_desc='
keyword = input('What do you want to eat ? ')
address = input('Where are you ? ')

url = 'https://www.yelp.com/search?find_desc='+ keyword + '&find_loc=' + address 
url

What do you want to eat ? pizza
Where are you ? new york


'https://www.yelp.com/search?find_desc=pizza&find_loc=new york'

In [16]:
# Checking if the url is "valid", the status code should be 200
response = requests.get(url)
response.status_code

200

In [17]:
# extract the html code from the web page = https://www.yelp.com/search?find_desc=Restaurants&find_loc=New+York,+NY&start=30
results_page = BeautifulSoup(response.content,'lxml')

In [ ]:
# creating a list of businesses with the html code of the business as elements of the list
businesses = results_page.find_all('a',class_ = "biz-name js-analytics-click")

In [ ]:
# selecting the first element (a particular restaurant in our case) of the list businesses and looking in the
#ignore the first element of the businesses list, it is an ad by yelp, regular list start from the second (index = 1)
# html code for its name
business = businesses[1]
name = business.find('span').get_text()

# getting the link to the yelp page this particular restaurant
link = 'https://www.yelp.com' + business.get('href')

In [ ]:
print(name,link)

In [ ]:
# extracting the html code of the web page associated with the first business of our list businesses
response_business = requests.get(link)
results_business_page = BeautifulSoup(response_business.content,'lxml')

In [ ]:
# looking in the html page of the restaurant for latitude and longitude coordinates of the restaurant
# the json object json_map contains these coordinates
json_map = results_business_page.find('div', class_ = "lightbox-map hidden").get('data-map-state')

# we can convert this json object to a python dictionnary using the following method
dict_map = json.loads(json_map)

# the relevant keys in this dict are "markers", "location" and "latitude" and "longitude"
latitude = dict_map["markers"][1]["location"]["latitude"]
longitude = dict_map["markers"][1]["location"]["longitude"]

In [ ]:
json_map

In [ ]:
print(latitude,longitude)

In [ ]:
# The category can be found in the 'a' tag under "span", class_ = 'category-str-list' tag, 
category= results_business_page.find("span", class_ = 'category-str-list').find('a').get_text()

In [ ]:
category

In [ ]:
# the price range ('$','$$','$$$', or '$$$$') can be found in the "span", class_ = 'business-attribute price-range' tag
# the price range is not always available, that is why we need to catch errors 
# if necessary (hence the try: except) structure

try:
    price_range = results_business_page.find("span", class_ = 'business-attribute price-range').get_text()
except:
    price_range = None

In [ ]:
print(price_range)

In [ ]:
# the ratings associated to a particular restaurant are in the "div", itemprop = 'aggregateRating' tag
# the rating value (between 0 and 5) can be accessed through ratings (= results_business_page.find("div", 
# itemprop = 'aggregateRating') at the tag "meta" and the attribute 'content'
# the number of reviews is available through ratings at the 'span' tag

ratings = results_business_page.find("div", itemprop = 'aggregateRating')
rating_value = ratings.find("meta").get('content')
review_count = ratings.find("span").get_text()

In [ ]:
print(rating_value,review_count)

In [ ]:
# the phone number can be accessed at the tag 'span',itemprop = "telephone"
# phone numbers are not always available so we need to catch errors using the try:... except: structure

try:
    phone_number = results_business_page.find('span',itemprop = "telephone").get_text().strip()
except:
    phone_number = None   

In [ ]:
print(phone_number)

In [ ]:
# We want to know if the restaurant is currently open: this information is 
# given at the 'span', class_ = "nowrap extra open" tag. This information is not always 
# available that is why we use the try: ... except:... structure

try:
    open_ = results_business_page.find('span', class_ = "nowrap extra open").get_text()
except:
    open_ = None

In [ ]:
print(open_)

In [ ]:
try:
    addr = results_business_page.find('strong', class_ = "street-address").get_text().strip()
except:
    addr = None
addr

In [ ]:
try:
    web = results_business_page.find('span', class_ = "biz-website js-biz-website js-add-url-tagging").find('a').get_text()
except:
    web = None
web

In [18]:
#want to put all infomations we want to get from one business into one funtion
#input: specific restaurant url link
#output: all details we tested above
def get_info(link):
    response_business = requests.get(link)
    results_business_page = BeautifulSoup(response_business.content,'lxml')
    
    try:
        addr = results_business_page.find('strong', class_ = "street-address").get_text().strip()
    except:
        addr = None

    try:
        json_map = results_business_page.find('div', class_ = "lightbox-map hidden").get('data-map-state')
        dict_map = json.loads(json_map)
        lat = dict_map["markers"][1]["location"]["latitude"]
        lon = dict_map["markers"][1]["location"]["longitude"]
    except: 
        json_map = None 

    try: 
        category = results_business_page.find("span", class_ = 'category-str-list').find('a').get_text()
    except:
        category = None
        
    try:
        price_range = results_business_page.find("span", class_ = 'business-attribute price-range').get_text()
    except:
        price_range = None
        
    try: 
        rating_value = results_business_page.find("div", {'itemprop' : "aggregateRating"}).find("meta").get('content')
    except: 
        rating_value = None
          
    try:
        phone_number = results_business_page.find('span',itemprop = "telephone").get_text().strip()
    except:
        phone_number = None
        
    try:
        open_ = results_business_page.find('span', class_ = "nowrap extra open").get_text()
    except:
        open_ = 'closed now'
        
    try:
        web = results_business_page.find('span', class_ = "biz-website js-biz-website js-add-url-tagging").find('a').get_text()
    except:
        web = None
    
    try:
        moreinfo = results_business_page.find_all('dl')
        delivery = "No"
        for i in moreinfo[1:]:
            try:
                attr = i.find('dt',{'class':"attribute-key"}).get_text().strip()
                yon = i.find('dd').get_text().strip()
                if attr == 'Delivery':
                    delivery = yon
                    break
            except:
                delivery = delivery

    except:
        delivery = None
        
    return(addr,category,price_range,rating_value,phone_number,open_,web,delivery,lat,lon)
    

In [ ]:
#Get a list of url links of all businesses (or maybe the first 30)
#so that we could compile a list of details on, for example, the first 30 search-result restaurants
link_list = []

for i in range(1,len(businesses)):
    business = businesses[i]
    link = 'https://www.yelp.com' + business.get('href')
    link_list.append(link)

In [ ]:
#Create a list of restaurant names
name_list=[]
for i in range(1,len(businesses)):
    business = businesses[i]
    name = business.find('span').get_text()
    name_list.append(name)

In [ ]:
#Create a list of all restaurants' information
info_list=[]

for i in range(len(link_list)):
    info_list.append(get_info(link_list[i]))

In [48]:
def get_businesses():
    global keyword
    keyword = input('What do you want to eat ? ')
    global address
    address = input('Where are you ? ')
    limit = input('How many results do you want? ')
    filters = input('Do you want to sort by \n 1: price low to high  \n 2: price high to low  \n 3: ratings? ')
    
    #first page - figure out numbers of results per page
    url = 'https://www.yelp.com/search?find_desc='+ keyword + '&find_loc=' + address 
    response = requests.get(url)
    try:
        results_page = BeautifulSoup(response.content,'lxml')
    except:
        print("we didn't get back result page from yelp correctly")
    noperpage = results_page.find('span',{'class':'pagination-results-window'}).get_text().strip()[10:12]
    noperpage = int(noperpage)
    businesses = results_page.find_all('a',{'data-analytics-label':"biz-name"})[1:]
    global cuisine_styles
    cuisine_styles = results_page.find_all('span',class_="category-str-list")
    
    #start loop from the second page to get enough number of restaurants
    loop_range = (int(limit)-1)//noperpage
    for pagination in range(1, loop_range+1):
        url = 'https://www.yelp.com/search?find_desc='+ keyword + '&find_loc=' + address + '&start=' + str(pagination*noperpage)
        response = requests.get(url)
        try:
            results_page = BeautifulSoup(response.content,'lxml')
        except:
            print("we didn't get back result page from yelp correctly")
        
        businesses.extend(results_page.find_all('a',{'data-analytics-label':"biz-name"})[1:])
    businesses = businesses[:int(limit)]
    
    #generate details by get_info and format in dataframe
    business_list = []
    for business in businesses:
        name = business.find('span').get_text()
        link = 'https://www.yelp.com' + business.get('href')
        addr,category,price_range,rating_value,phone_number,open_,web,delivery,lat,lon= get_info(link)
        business_list.append([name,addr,open_,price_range,rating_value,category,phone_number,web,delivery,lat,lon])
    df = pd.DataFrame(business_list[:int(limit)+1], columns = ['name','address','open or closed','price range','rating value','category','phone','website','delivery service','lat','lon'])
    
    #sortby...
    pattern = r'rat'
    match = bool(re.search(pattern, filters))
    if match or filters == '3':
        df = df.sort_values(by='rating value',ascending=False)
    
    elif filters == 'price low to high' or filters == '1':
        df['length'] = df['price range'].apply(lambda x: len(x))
        df = df.sort_values(by='length',ascending=True)
        df = df.drop('length', axis=1)
        
    elif filters == 'price high to low' or filters == '2':
        df['length'] = df['price range'].apply(lambda x: len(x))
        df = df.sort_values(by='length',ascending=False)
        df = df.drop('length', axis=1)
    
    return(df)

In [49]:
res = get_businesses()

What do you want to eat ? pizza
Where are you ? new york
How many results do you want? 15
Do you want to sort by 
 1: price low to high  
 2: price high to low  
 3: ratings? 3


In [50]:
# print(filters)
# print(re.search(pattern, filters))
res

,name,address,open or closed,price range,rating value,category,phone,website,delivery service,lat,lon
7,L’industrie Pizzeria,"254 S 2nd StBrooklyn, NY 11211",Open now,$,5.0,Pizza,(718) 599-0002,lindustriebk.com,Yes,40.711584,-73.957871
0,Juliana’s Pizza,"19 Old Fulton StBrooklyn, NY 11201",Open now,$$,4.5,Pizza,(718) 596-6700,julianaspizza.com,No,40.702615,-73.993416
1,Macoletta,"28-15 24th AveAstoria, NY 11102",Open now,$$,4.5,Pizza,(718) 777-4992,macoletta.com,Yes,40.773242,-73.916415
3,Prince Street Pizza,"27 Prince StNew York, NY 10012",Open now,$,4.5,Pizza,(212) 966-4100,princestpizzanewyork.com,Yes,40.722909,-73.994486
4,Brooklyn Pizza Masters,"1055 1st AveNew York, NY 10022",Open now,$,4.5,Pizza,(646) 669-7757,None,Yes,40.758655,-73.963018
5,800 Degrees Woodfired Kitchen,"1 E 33rd StNew York, NY 10016",Open now,$$,4.5,Pizza,(212) 660-6419,800degrees.com,Yes,40.747652,-73.984506
6,Sauce Pizzeria,"345 E 12th StNew York, NY 10003",Open now,$$,4.5,Pizza,(646) 983-4007,None,Yes,40.730430,-73.983944
11,Mama Rosa’s Brick Oven Pizzaria Restaurant,"77-09 37th AveJackson Heights, NY 11372",Open now,$,4.5,Pizza,(347) 649-1862,None,Yes,40.749511,-73.888552
12,Baker’s Pizza,"754 10th AveNew York, NY 10019",Open now,$$,4.5,Pizza,(917) 262-0722,bakerspizzanyc.com,Yes,40.765425,-73.990923
2,Lombardi’s Pizza,"32 Spring StNew York, NY 10012",Open now,$$,4.0,Pizza,(212) 941-7994,firstpizza.com,Yes,40.721637,-73.995721


In [ ]:
import folium
m = folium.Map(location=[40.8075355, -73.9625727],zoom_start=14)

In [ ]:
# a marker is created for any restaurant: red for critical situation and orange for not critical
for k in range(len(res)):
    folium.Marker([res['lat'].iloc[k],res['lon'].iloc[k]],popup=res['name'].iloc[k]).add_to(m)
m

In [19]:
cuisine_styles = results_page.find_all('span',class_="category-str-list")
#print(cuisine_styles)
# print(cuisine_styles[4].find('a').get_text())

styles = []
for i in range(len(cuisine_styles)):
    style = [x.get_text() for  x  in cuisine_styles[i].find_all('a')]
    styles.extend(style)

styles

['Pizza',
 'Italian',
 'Pizza',
 'Pizza',
 'Pizza',
 'Italian',
 'Pizza',
 'Pizza',
 'Italian',
 'Pizza',
 'Pizza',
 'Pizza',
 'Italian',
 'Pizza',
 'Pizza',
 'Italian',
 'Beer Bar',
 'Pizza',
 'Pizza',
 'Pizza',
 'Diners',
 'Pizza',
 'Italian',
 'Pizza',
 'Pizza',
 'Pizza',
 'Wine Bars',
 'Beer Bar',
 'Pizza',
 'Pizza',
 'Italian',
 'Pizza',
 'Italian',
 'Pizza',
 'Bars',
 'American (New)',
 'Pizza',
 'Pizza',
 'Italian',
 'Pizza',
 'Italian',
 'Wine Bars',
 'Pizza',
 'Bars',
 'Italian',
 'Pizza',
 'Pizza',
 'Vegan',
 'Italian',
 'Pizza',
 'Salad',
 'Italian',
 'Pizza',
 'Pizza']

In [47]:
# from several random keyword searching, we found that only the first two cuisine styles have significant counts
# e.g. out of the 30 results, the 3rd most common style appears around only 5 times
count = collections.Counter(styles).most_common(2)
print(count)

# Get a new keyword
if keyword == count[0][0].casefold():
    keyword2 = count[1][0].casefold()
else:
    keyword2 = count[0][0].casefold()

print(keyword2)

[('Pizza', 31), ('Italian', 13)]
italian


In [51]:
# Give users an opportunity to see recommendations relevant cuisine
# e.g. if originally searched for "pizza" but did not like the recommendations
# this function returns 10 more recommendations of "Italian" restaurants
def further_rec():
    reaction = input('Are you satisfied with your recommendations? (yes/no)')
    
    if reaction == 'yes':
        return 'Bon Appetit!'
    else:
        # Moved cuisine_styles to get_business() and make it global for access in this function
        styles = []
        for i in range(len(cuisine_styles)):
            style = [x.get_text() for  x  in cuisine_styles[i].find_all('a')]
            styles.extend(style)

        count = collections.Counter(styles).most_common(2)
        if keyword == count[0][0].casefold():
            keyword2 = count[1][0].casefold()
        else:
            keyword2 = count[0][0].casefold()

        url = 'https://www.yelp.com/search?find_desc='+ keyword2 + '&find_loc=' + address
        # new keyword but same address as from get_business()
        response = requests.get(url)
        try:
            results_page = BeautifulSoup(response.content,'lxml')
        except:
            print("we didn't get back result page from yelp correctly")
                
        businesses = results_page.find_all('a',{'data-analytics-label':"biz-name"})[1:]
        business_list = []
        for business in businesses:
            name = business.find('span').get_text()
            link = 'https://www.yelp.com' + business.get('href')
            addr,category,price_range,rating_value,phone_number,open_,web,delivery,lat,lon= get_info(link)
            business_list.append([name,open_,price_range,rating_value,category,phone_number,web,delivery,lat,lon])
                                 
        df = pd.DataFrame(business_list[:11], columns = ['name','open or closed','price range','rating value','category','phone','website','delivery service','lat','lon'])
        print('Take a look at the following 10 restaurants!')                                   
        return df

In [52]:
fr = further_rec()
fr

Are you satisfied with your recommendations? (yes/no)no
Take a look at the following 10 restaurants!


,name,open or closed,price range,rating value,category,phone,website,delivery service,lat,lon
0,Angeletto Restaurant,Open now,$$,4.5,Italian,(212) 235-1227,angeletto.com,No,40.753188,-73.969365
1,L’Artusi,Open now,$$$,4.5,Italian,(212) 255-5757,lartusi.com,No,40.733714,-74.005068
2,Patavini,Open now,$$$,4.5,Pasta Shops,(845) 875-4660,patavini.com,Yes,40.767281,-73.981349
3,Malatesta Trattoria,Open now,$$,4.0,Italian,(212) 741-1207,None,No,40.732849,-74.008439
4,Trattoria Trecolori,Open now,$$,4.0,Italian,(212) 997-4540,trattoriatrecolori.com,Yes,40.759861,-73.986791
5,Botte Bar,Open now,$$,4.5,Cocktail Bars,(718) 433-9079,bottebar.com,Yes,40.763354,-73.928050
6,Nonna Beppa Soho,Open now,$$,4.0,Italian,(917) 639-3996,nonnabeppa.com,Yes,40.725491,-74.007413
7,La Lanterna Di Vittorio,Open now,$$,4.0,Italian,(917) 639-3236,lalanternacaffe.com,No,40.730709,-74.000147
8,Rubirosa,Open now,$$,4.0,Italian,(212) 965-0500,rubirosanyc.com,Yes,40.722753,-73.996202
9,Sapori D’Italia,Open now,$$,4.5,Italian,(212) 714-2113,saporinyc.com/#!home,Yes,40.717331,-73.998590


In [ ]:
#workspace with saved results page
url = 'https://www.yelp.com/search?find_desc=lobster&find_loc=new+york&start=60'
response = requests.get(url)
try:
    results_page = BeautifulSoup(response.content,'lxml')
except:
    print("we didn't get back result page from yelp correctly")
  

In [ ]:
businesses = results_page.find_all('a',{'data-analytics-label':"biz-name"})[1:]
businesses.extend(businesses)
businesses

In [ ]:
# Yelp is also providing API keys to do some web scraping on there website. We created our own API key to be able
# to do some web scraping when limited by the direct html scraping method we were using before

with open('./YelpAPIKeys.txt','r') as f:
    count = 0
    for line in f:
        if count == 0:
            CLIENT_ID = line.strip()
        if count == 1:
            API_KEY = line.strip()
        count+=1

In [ ]:
print(CLIENT_ID,API_KEY)

In [ ]:
API_HOST = 'https://api.yelp.com' # this is the API url header
SEARCH_PATH = '/v3/businesses/search' # this is the path for an API request to find businesses
BUSINESS_PATH = '/v3/businesses/'  # this is the path to get data for a single business

In [ ]:
# this function creates a list of businesses, given an API key and a location like New York,...

def get_restaurants(api_key,location,number=50):
    import requests
    
    #First we get the access token
    #Set up the search data dictionary
    search_data = {
    'term': "restaurant",
    'location': location.replace(' ', '+'),
    'limit': number
    }
    url = API_HOST + SEARCH_PATH
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }
    response = requests.request('GET', url, headers=headers, params=search_data).json()
    businesses = response.get('businesses')
    return businesses

In [ ]:
get_restaurants(API_KEY,"New York",number=5)

In [ ]:
restaurants_data

In [ ]:
# This function creates a dataframe based on the data that we extracted from yelp website, 
# using the function get_restaurants

def data_output(restaurants_data):
    df = pd.DataFrame(columns = ['name','is_closed','latitude','longitude','price','rating','categories','phone'])
    for i in range(len(restaurants_data)):
        name = restaurants_data[i]['name']
        is_closed = restaurants_data[i]['is_closed']
        latitude = restaurants_data[i]['coordinates']['latitude']
        longitude = restaurants_data[i]['coordinates']['longitude']
        try:
            price = restaurants_data[i]['price']
        except:
            price = None
        rating = restaurants_data[i]['rating']
        categories = []
        for categorie in restaurants_data[i]['categories']:
            categories.append(categorie['title'])
        phone = restaurants_data[i]['phone']
        
        df.loc[i] = [name,is_closed,latitude,longitude,price,rating,categories,phone]
    df.set_index('name', inplace = True)
    return(df)

In [ ]:
res = data_output(restaurants_data)

In [ ]:
res

In [ ]:
def web_scrapingYelp():
    import requests
    
    address = input('Where are you ? ')
    limit = input('How many results do you want? ')

    search_data = {
    'term': "restaurant",
    'location': address.replace(' ', '+'),
    'limit': limit
    }
    url = API_HOST + SEARCH_PATH
    headers = {
        'Authorization': 'Bearer %s' % API_KEY,
    }
    response = requests.request('GET', url, headers=headers, params=search_data).json()
    businesses = response.get('businesses')
    
    df = pd.DataFrame(columns = ['name','is_closed','latitude','longitude','price','rating','categories','phone'])
    for i in range(len(businesses)):
        name = businesses[i]['name']
        is_closed = businesses[i]['is_closed']
        latitude = businesses[i]['coordinates']['latitude']
        longitude = businesses[i]['coordinates']['longitude']
        try:
            price = businesses[i]['price']
        except:
            price = None
        rating = businesses[i]['rating']
        categories = []
        for categorie in businesses[i]['categories']:
            categories.append(categorie['title'])
        phone = businesses[i]['phone']
        
        df.loc[i] = [name,is_closed,latitude,longitude,price,rating,categories,phone]
    
    df.set_index('name', inplace = True)
    
    return(df)

In [ ]:
web_scrapingYelp()
